In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

In [2]:
spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-1")\
    .config("spark.yarn.access.hadoopFileSystems","s3a://demo-aws-1/")\
    .config("spark.hadoop.yarn.resourcemanager.principal",os.getenv("HADOOP_USER_NAME"))\
    .getOrCreate()
#.config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
#    .config("spark.executor.instances", 4)\
#    .config("spark.executor.cores", 4)\

In [3]:
df = spark.read.option('inferschema','true').csv(
  "s3a://demo-aws-1/user/pauldefusco/LoanStats_2015_subset.csv",
  header=True,
  sep=',',
  nullValue='NA'
)

In [4]:
spark.sql("show databases").show()

+--------------------+
|        databaseName|
+--------------------+
|  airline_ontime_orc|
|airline_ontime_pa...|
|        airline_temp|
|         airlinedata|
|                dbgr|
|             default|
|  information_schema|
|     insurance_fraud|
|       prescribing_o|
|       prescribing_p|
|     prescribing_p_e|
|prescribing_p_e_d...|
|          retaildemo|
|         shlomi_test|
|                 sys|
|                test|
|             test_dl|
|            testuser|
|            tutorial|
|         ukcrime_dev|
+--------------------+
only showing top 20 rows



In [5]:
df.write.format('parquet').mode("overwrite").saveAsTable('default.spark_lineage_table_origin')

In [6]:
df.show(1)

+--------------+--------------------+----------+--------+----------+----------------+----------------+-----------+--------------+-------+------------------------+--------------------------+-----------+-----------+----+-----+---------+----------------+----------+---------+-----------+---------------+-----+--------------+---+-------+-------------------+------+------------+--------------+-----------+--------+--------+---------+-----------+----------+---------+------------------+--------------------+---------------------+--------------+--------+----------------------+---------------------------+----------------------+------------------+--------------------+------------------------+---------------------+------------------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+----------------+------------+------------------+------------------+--------+-----------+-----------+-----------+------

In [7]:
df1 = spark.sql("select annual_inc, acc_now_delinq, acc_open_past_24mths, addr_state, all_util, annual_inc_joint from default.spark_lineage_table_origin limit 50")

In [8]:
df2 = spark.sql("select annual_inc, emp_length, emp_title, funded_amnt, funded_amnt_inv from default.spark_lineage_table_origin limit 59")

In [9]:
df1.show()

+----------+--------------+--------------------+----------+--------+----------------+
|annual_inc|acc_now_delinq|acc_open_past_24mths|addr_state|all_util|annual_inc_joint|
+----------+--------------+--------------------+----------+--------+----------------+
|   65000.0|             0|                   8|        KS|      42|            null|
|  150000.0|             0|                   1|        VT|      34|            null|
|   75000.0|             0|                   8|        IN|      50|            null|
|  175000.0|             0|                   7|        OH|      71|            null|
|   44000.0|             0|                   1|        FL|      91|            null|
|  104000.0|             0|                   1|        OH|      52|            null|
|   38000.0|             0|                   4|        OH|      30|            null|
|  109000.0|             0|                   2|        VA|      57|            null|
|   70000.0|             0|                   3|      

In [10]:
df2.show()

+----------+----------+--------------------+-----------+---------------+
|annual_inc|emp_length|           emp_title|funded_amnt|funded_amnt_inv|
+----------+----------+--------------------+-----------+---------------+
|   98500.0|   9 years|Senior Credit Ana...|      24000|        24000.0|
|   50000.0|    1 year| Ocuppational The...|       9000|         9000.0|
|   49920.0|   9 years|             Foreman|      17000|        17000.0|
|   95028.0|       n/a|                    |      24225|        24225.0|
|   50000.0|    1 year| Services Specialist|      16000|        16000.0|
|   68000.0|       n/a|                    |      15000|        15000.0|
|   24000.0|    1 year|Transport Specialist|       6000|         6000.0|
|   35000.0|  < 1 year|Medical technologist|      12250|        12250.0|
|   40000.0|   3 years|  Nursing Attendant |       4800|         4800.0|
|   55000.0|    1 year|  Concrete Placement|      16000|        16000.0|
|   96000.0|   7 years| Eucation Specialist|      1

In [11]:
df1.write.format('parquet').mode("overwrite").saveAsTable('default.spark_lineage_table_left')

In [12]:
df2.write.format('parquet').mode("overwrite").saveAsTable('default.spark_lineage_table_right')

In [13]:
df3 = spark.sql("select A.annual_inc, B.emp_length, B.emp_title from default.spark_lineage_table_left A inner join default.spark_lineage_table_right B on A.annual_inc = B.annual_inc")


In [14]:
df3.count()

33

In [15]:
df3.write.format('parquet').mode("overwrite").saveAsTable('default.spark_lineage_table_joined')

In [17]:
df4 = spark.sql("CREATE TABLE default.spark_lineage_final AS SELECT * FROM default.spark_lineage_table_joined")

In [18]:
spark.sql("CREATE TABLE default.spark_lineage_test AS SELECT * FROM default.spark_lineage_table_joined")

DataFrame[]

In [19]:
spark.sql("CREATE TABLE default.spark_lineage_new AS SELECT * FROM default.spark_lineage_final")

DataFrame[]

In [ ]:
#spark.sql("CREATE TABLE default.spark_test AS SELECT * FROM default.spark_lineage_final")